In [1]:
from openai import OpenAI

client = OpenAI()

In [7]:
functions = [
    {"type": "function",
     "function": {"name": "similarity_search",
     "description": "Use this function to answer questions about specific topics. Pass all meaningful words as a query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "search_term": {
                        "type": "string", 
                        "description": "The meaningful string, stripped of stopwords and uninformative words."
                        },                  
                    },
                "required": ["a search_term"]}}}]

similarity_search = "drunken night"

In [9]:
res = client.chat.completions.create(model="gpt-3.5-turbo-0613", # "gpt-4",
                            messages=[
                                {"role": "user", "content": f"What would johannes say about: {similarity_search}?"}],
                            max_tokens=1000,
                            tools=functions)

In [12]:
res.choices[0].message.tool_calls

[ChatCompletionMessageToolCall(id='call_pN7yGn8svAKdRN2NXqLJu40D', function=Function(arguments='{\n  "search_term": "drunken night"\n}', name='similarity_search'), type='function')]

## The supernova copilot as a model

note that i should force the model into funciton calls each time. 

In [ ]:
from openai import OpenAI
import json, time

client= OpenAI()

def openai_stream(search_term):  
    response = client.chat.completions.create(
                                model="gpt-4-1106-preview",
                                messages=[{"role": "user", "content": f"What would johannes say about: {search_term}?"}], 
                                stream=True,
                                temperature=0.4,
                                max_tokens=1024,
                                # tool_choice={"type: "function", "function": {"name": "similarity_search"}},
                                tools=functions,
                                ) # type: ignore
    return response

def stream_chunks(text):
    words = text.split(" ")
    for word in words:
        time.sleep(0.1)
        yield word + ' ' # was json

def openai_call(search_term:str):
    arguments = ""
    name = ""
    tool_call_id = ""
    try: 
        for res in openai_stream(search_term):
            choice = res.choices[0]
            delta = choice.delta
            if delta.tool_calls:
                for tool_call in delta.tool_calls:
                    # if multiple tool calls i'd have to deal with lists of these:
                    name += tool_call.function.name or ''
                    arguments += tool_call.function.arguments or ''
                    tool_call_id += tool_call.id or ''
                    
            if choice.finish_reason and 'tool_calls' in choice.finish_reason:
                print("function call: %s %s", name, arguments, id)
                args = json.loads(arguments)
                data = None
                
                # function_to_call = globals().get(name)
                if 'similarity_search' in name:
                    print('similarity_search is in name')
                    
                    data = similarity_search(**args)
                    if data:
                        yield from interpreting_data(tool_call_id, search_term, name, args, data) 
                else: 
                    print('similarity_search is not in name')
                
    except Exception as e:
        print("An exception: ", e.args, e.with_traceback)
        yield from stream_chunks(f"Sorry, I'm having some trouble. I forwarded the error to our tech team. {e}")
        
def interpreting_data(tool_call_id, search_term:str, name:str, args:str, data:str): # generator
    full_response = ''
    completion_res = client.chat.completions.create(
        model="gpt-4-1106-preview",
        temperature=0,
        messages=[{"role": "user", "content": f"Wat schrijft de auteur over {search_term}?"},
                  {"role": "assistant", "content": 'null', 'tool_calls': [{'id': tool_call_id, 'function': {"name": name,'arguments': str(args)}, 'type': 'function'}]},
                  {"tool_call_id": tool_call_id, "role": "tool", "name": name, "content": data} # type: ignore
                  ],
        stream=True,
        tools=functions) # type: ignore
    try:
        for res in completion_res:
            content_chunk = res.choices[0].delta.content
            if content_chunk:
                full_response += content_chunk
                yield content_chunk # was json   
    
    except Exception as e:
        raise Exception(f"interpreting_data returned an unexpected error: {e}") from e
    
# semantic search
def similarity_search(search_term, results = 3):
    print("function call with search_term", search_term)
    
    df = pd.read_pickle("df_topic_model_all_v2")
    df = df[df['mask']].copy()
    embeddings = df.normal_embeddings.tolist()
    user_text_embedding = sentence_model.encode(sentences=[search_term])
    cosine_similarities = cosine_similarity(user_text_embedding, embeddings)
    df['similarity'] = cosine_similarities.flatten()
    df_pickle_clean_sorted = df.sort_values(by='similarity', ascending=False)
    json = df_pickle_clean_sorted[['texts', 'similarity']].iloc[:results].to_json(orient='records')
    return json

In [ ]:
for i in openai_call('een diepe crisis'):
    print(i, end='')

function call: %s %s similarity_search {"search_term":"een diepe crisis"} <built-in function id>
similarity_search is in name
function call with search_term een diepe crisis
De auteur schrijft over een diepe crisis in verschillende contexten. Hier zijn enkele fragmenten die de auteur heeft geschreven over situaties die kunnen worden geïnterpreteerd als een diepe crisis:

1. De auteur beschrijft een gevoel van reddeloosheid en overweldiging door bureaucratie en administratieve rompslomp in sociale instellingen zoals de Hulpkas of VDAB. Er wordt een beeld geschetst van een gebrek aan arbeidsvreugde en een eindeloze wildgroei van formulieren en procedures.

2. Er wordt gesproken over een persoonlijke crisis, waarbij de auteur een behoefte voelt om zich uit te drukken en zijn ziel uit te smeren over woorden. De auteur voelt zich echter beklemd en geïsoleerd door de confrontatie met anderen die zijn woorden beoordelen op autoriteit in plaats van inhoud.

3. De auteur reflecteert op de spann

## this wasn't clear in the docs:

- function calling vs tool_calls

```py
# function calls need this before the "role": "function"
{"role": "assistant", "content": 'null', "function_call": {"name": name, "arguments": args}}


# tool calls need this line
{"role": "assistant", "content": 'null', 'tool_calls': {'id': tool_call_id, 'function': { 'arguments': args, 'type':'function'}}}
```


this is an object of the class a ChatCompletionMessageToolCall, as seen when inspecting the output of a (non-streaming) `response.choices[0].message` object:

```py

from openai.types.chat.chat_completion_message import ChatCompletionMessageToolCall


tool_call = ChatCompletionMessageToolCall()
```
## This shows (some) of the required inputs when instantiating the class as required inputs

Such as id, function, etc.

>  tool_calls=[ChatCompletionMessageToolCall(id='call_MgpkBPSnKyt8jtcOvESazlgF', function=Function(arguments='{"location":"San Francisco, CA"}', name='get_current_weather'), type='function')]

# embedding model on cloud run?

Hugging face advertises the API inference. This does similarity search like this:

In [2]:
import os
API_TOKEN = os.getenv('HUGGINGFACE')

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
headers = {"Authorization": f"Bearer {API_TOKEN}"}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()
	
output = query({
	"inputs": {
		"source_sentence": "That is a happy person",
		"sentences": [
			"That is a happy dog",
			"That is a very happy person",
			"Today is a sunny day"
		]
	},
})

output

Answer: i'd have to upload and download the entire array each time and basically redo all embeddings each time. Probably not cost effective. 

# Other issues

In [14]:
json_bs = [{"texts":"Ik stond in monaco naast mijn nieuwe Porsche GT3 toen hij met z\u2019n afstandsbediening naar mijn wagen kwam gelopen. Piebiep-biep. Het was ook Porschesleutel, ik herkende hem. \u201cHah,\u201d lachte hij. En hij liep lachend verder, naar de auto die voor d emijne stond geparkeerd. Het was mij nog niet echt opgevallen, maar het was krek dezelfde. Ik lachte naar hem: je hebt echt krek dezelfde als ik. Ja, zei hij. Hij was een visiounair een groot architectenbureau, zei hij. X was zijn naam. Hij was nu bezig aan een brochure over scholenbouw. Hij vertelde me er alles over. Over zijn iPads, zijn motors, zijn beeldschone vrouw, zijn onverzadigbare minnares. We hebben liters champagne achterover geklutst, daar samen op de dijk. Heerlijk.\n","similarity":0.3990037441},{"texts":"Review van Geert Maks In Europa\nWaar waren we ook alweer gebleven?\nGisterenavond zag ik de laatste aflevering van Geert Maks magnum opum In Europa. Na zijn boek van 852 wonderlijke bladzijden was de documentaire met zijn 35 afleveringen en 18 en een half uur aan beeldmateriaal een leuke aanvulling.\nOm mijn algemene indruk in \u00e9\u00e9n woord weer te geven: gepriviligeerd. Ik voelde me gepriviligeerd een inkijk te hebben mogen nemen in zijn perspectief. Geert Mak is een oprechte, bescheiden vakman en dat werkt inspirerend.\nIn 2004 kwam het boek uit. In Nederland kreeg hij een eredoctoraat (hij is geen historicus maar jurist van opleiding). Het werd in 14 talen vertaald en was een groot succes in Nederland, Belgi\u00eb, Duitsland, Engeland en Frankrijk. De (Nederlandstalige) serie kwam van 2007 tot 2009 op tv en is online te bekijken op:\nDit verklaart wellicht al een deel van die algemene indruk. Maar zijn ware verdienste ligt ongetwijfeld in de beeldenrijke inkleuring die hij aan het tijdvak heeft weten te geven. Geert Mak is een verhaalverteller, en de spreekwoordelijke beelden staan in mijn geheugen gegrift.\nIn de documentaire ging de voorkeur veel meer uit naar de perspectieven van mensen die het meemaakten. Daardoor het succes vaak kwam af te hangen van het charisma van de personen die voor de lens werden opgevoerd.\nZo is er de aflevering over de foto van het \u201cvolksduits\u201d meisje in Polen wellicht het best geslaagd in dat opzet. Het brengt met een zeer herkenbaar referentiepunt die vergeten geschiedenis van het Generaalgouvernement in Polen weer.\nHet boek volgt een reistraject. Telkens behandelt hij een steeds variabele historische tijdspanne in contrast met zijn reisimpressies. Elk deel begint dan ook met een kaartje waarop het stuk van de reisweg wordt getoond, en vervolgens wordt elke plek in kleine, makkelijk verteerbare stukjes beschreven.\nDe historische scharnierpunten komen echter ruimschoots aan bod. De geschiedenis wordt helemaal terug tot leven gewekt door gebruik te maken van memoires en dagboekfragmenten. Ik heb mijn vingers afgelikt na de honderden pagina's over de roerige interbellumjaren in Berlijn. Een paar weken geleden toen ik Berlijn opnieuw bezocht, bekeek ik de stad met nieuwe ogen. Berlijn is d\u00e9 stad van de 20ste eeuw. Elke beschreven gebeurtenis loopt over van emotie. Dat doet hij door zijn subjecten goed uit te kiezen. En eens zijn dat de beslissingen van politici, dan weer de opstandige volkeren, maar ook bijvoorbeeld het historisch materialisme dat destijds nog een wereldrevolutie in petto had.\nHij gebruikt ook een aantal literaire trucs om de spanning op te bouwen, de lezer mee te slepen, en elk deeltje met een knaller te eindigen.\nMaar ik voelde me in het bijzonder gepriviligeerd omdat Maks perspectief zo dicht bij huis is, waardoor ik me makkelijk met zijn sensibiliteiten kon identiferen. Geschiedenis wordt doorgaans verteld door de overwinnaars, zoals het gezegde gaat. In dezen waren dat de Amerikanen. Zij waren de winnaars van de Tweede Wereldoorlog en de Koude Oorlog, en zij waren vanaf de tweede helft van de 20ste eeuw de dominante economische, politieke en culturele macht. Als men er vandaag op terugkijkt, faalt men zich vaak rekenschap af te leggen van dergelijke subtiliteiten, en dat de kijk op verleden en toekomst is mee veranderd met die verschuiving.\nMak legt een diepgevoeld humanisme aan de dag en een nederigheid tegenover zijn opgave, een deugd die je elk schrijver kan toewensen. Niet enkel is hij een trots voorvechter van de welvaartstaat, maar daarnaast is hij een multiculturalist. Het feit dat zijn ouders in de jaren 50 een Hongaarse vluchteling in huis hadden genomen, zal er wellicht toe bijgedragen hebben. Zijn sympathi\u00ebn reiken voorbij landsgrenzen, voorbij IJzeren Gordijnen, en dat voel je aan zijn proza.\nTen slotte was Mak ook een Amsterdamse student in het revolutiejaar 68, een generatiestrijd die ook de generatie van mijn ouders sterk be\u00efnvloedde (denk aan het holisme van de New Age, de alternatieve geneeskunde, flower power en al de rest die daaruit voortkwam). Een solidairdere samenleving heeft het ons niet opgeleverd, wel een samengevoegd Europa, en dat is een begin. Van wat ik van hieruit kan zien, is een globaal verbond van welvaartstaten de horizon. Dat is waar we heen moeten, en we hebben onze geschiedenis als gids en leidraad.\n","similarity":0.3735808134},{"texts":"Liefde baant in elkeen haar weg om ongehinderd door heggen en omheiningen, boven de daken van het dorp uitgeschreeuwd te worden. In het bevlogen moment van de verliefdheid wordt het nihilisme vertrappeld in de stormloop naar het hoogste punt om van te schreeuwen. Geen aandacht schenkt men nog aan de machtsverdelingen. De liefde overschreeuwd alles.\n","similarity":0.371294558}]
# json.dumps(json_bs, indent=6)

In [6]:
zin = "Topic clusterIn het kosmische dansfeest zweeft de interstellaire DJ die de nevels van de tijd doorklieft, terwijl de melodieën van oneindige genres zintuigen prikkelen en sterrenstelsels in harmonisch bewegen brengen. Meer weten? Klik hieronder.Generate cluster snippet"

import re

re.search(r'Topic cluster(.*?) Meer weten?', zin).group(1)

'In het kosmische dansfeest zweeft de interstellaire DJ die de nevels van de tijd doorklieft, terwijl de melodieën van oneindige genres zintuigen prikkelen en sterrenstelsels in harmonisch bewegen brengen.'

In [15]:
json_string = """[{"texts":"Bij dit laatste dacht ik sterk aan Pierre zijn gevoel voor stoerheid, met zijn aggressief werpen van zijn spullen, alsof hij er niet om geeft, en vooral zijn nihilistische, grove relativering van alles wat hij op zijn weg tegenkomt: \u201cIls font n'importe quoi\u201d, \u201c\u00e7a me fait chi\u00e9\u201d, \u201cputain, \u00e7a c'est de la merde,\u201d \u201crien \u00e0 foudre,\u201d \u201cil faut crever quoi.\u201d Het \u2018wij\u2019 is een genadeloze macht. Het lacht de zwakkeren uit (net als Alex) en het geeft om alles wat sensatie brengt, alles wat de retoriek pittiger maakt.\nBij een speech van Nicolas Dupont-Aignan in de \u00e9cole de commerce (hier in Toulouse) dacht ik na over het feit dat alle vragen, alle problematiseringen kennelijk gericht zijn op pragmatiek. Maar wat hier werkelijk op tafel ligt zijn niet de pragmatische redeneringen, deze probleemoplossingen, maar wel de Franse identiteit: \u201choezeer willen we het karakter van onze staat vrijwaren?\u201d Met andere woorden, het is alsof iedereen het weet, iedereen het mee-veronderstelt, maar niemand het durft te vragen: is de protectionistische reflex t.o.v. taal en traditie geen kwaal op zich? \u2018Op zich\u2019 wordt het niet in overweging genomen.\nZoals Dupont-Aignan zei: \u201cEuropa bestaat uit landen die samenwerken.\u201d Landen worden beschouwd als entiteiten met een specifieke, onvervreemdbare eigenheid.\nHet was een Gaullist.\n","similarity":0.8267167948},{"texts":"Post-it\u2019s:\nFrankrijk is een voortrekker geweest in de constructie van het natiestaten-model. Ook het centraal gezag vanuit Parijs had grote weerklank in de rest van de wereld.\n","similarity":0.8131709847},{"texts":"columnisme\n","similarity":0.812909394}]"""

In [24]:
import json
json.dumps(json.loads(json_string.replace('\n', '\\n')))

'[{"texts": "Bij dit laatste dacht ik sterk aan Pierre zijn gevoel voor stoerheid, met zijn aggressief werpen van zijn spullen, alsof hij er niet om geeft, en vooral zijn nihilistische, grove relativering van alles wat hij op zijn weg tegenkomt: \\u201cIls font n\'importe quoi\\u201d, \\u201c\\u00e7a me fait chi\\u00e9\\u201d, \\u201cputain, \\u00e7a c\'est de la merde,\\u201d \\u201crien \\u00e0 foudre,\\u201d \\u201cil faut crever quoi.\\u201d Het \\u2018wij\\u2019 is een genadeloze macht. Het lacht de zwakkeren uit (net als Alex) en het geeft om alles wat sensatie brengt, alles wat de retoriek pittiger maakt.\\nBij een speech van Nicolas Dupont-Aignan in de \\u00e9cole de commerce (hier in Toulouse) dacht ik na over het feit dat alle vragen, alle problematiseringen kennelijk gericht zijn op pragmatiek. Maar wat hier werkelijk op tafel ligt zijn niet de pragmatische redeneringen, deze probleemoplossingen, maar wel de Franse identiteit: \\u201choezeer willen we het karakter van onze s

how to run my app in ubuntu:
```py
# http://localhost:5000/ or http://johannes.vc/

# wsl -d Ubuntu
# /mnt/c/Users/johan/Documents/GitHub/johannes.vc
# export PATH=$PATH:/home/johannesvc/.local/bin
# gunicorn -w 4 'website:create_app()'

# http://127.0.0.1:8000
# ```

In [17]:
letters = 0
for word in json_string.split(' '):
    for letter in word:
        letters += 1
letters

1388